# Week 6 Day 2: Linear Algebra

## Objectives:

* Perform basic linear algebra manipulations
* Solve a realistic problem

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

## Matrix multiplication

All operations on an array are element-wise. Numpy used to have a "matrix" mode, where all operations were "matrix-wise"; that is, something like `*` would do matrix multiplication instead of element-wise multiplication. This was ugly and messy, and has been replaced in Python 3.5+ with a matrix multipy operator, `@`. (Older Python: Use `.matmul()` or `.dot()`.)

Let's first look at the diminsion rules for matrix multiplicaiton:

```
[a, b] @ [b, c] = [a, c]
```

In [ ]:
(np.ones([3, 4]) @ np.ones([4, 5])).shape

The "inner" diminsions go away. This works for ND arrays, too:

```
[a] @ [a] = scalar
```

In [ ]:
(np.ones([4]) @ np.ones([4])).shape

One of the two is allowed to have more than 2 dimensions, in which case it behaves like "stacks" of arrays:

```
[a,b,c] @ [c,d] = [a,b,d]
```

In [ ]:
(np.ones([2, 3, 4]) @ np.ones([4, 5])).shape

Normal "prepend 1" broadcasting rules apply.

In [ ]:
np.array([1, 2, 3]) @ np.array([[1, 2, 3]]).T

### Power user: Einstein summation notation

You can use [Einstein summation notation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html) for full control:

In [ ]:
a = np.arange(25).reshape(5, 5)

In [ ]:
np.trace(a)

In [ ]:
np.einsum("ii", a)

In [ ]:
a.T

In [ ]:
np.einsum("ji", a)

In [ ]:
a @ a

In [ ]:
np.einsum("ij,jk", a, a)

In [ ]:
np.sum(a * a)

In [ ]:
np.einsum("ij,ij", a, a)

In [ ]:
np.einsum("ij->", a ** 2)

## Linear algebra

Let's look at a bit of Linear algebra now.

We'll solve the equation:
$$
\mathbf{b} = A \mathbf{x}
$$

Which has the solution:

$$
\mathbf{x} = A^{-1} \mathbf{b}
$$

In [ ]:
b = np.array([1, 2, 3])
print(b)

In [ ]:
A = np.array([[1, 2, 3], [22, 32, 42], [55, 66, 100]])
print(A)

In [ ]:
np.linalg.inv(A) @ b

Note that for these equations, 1D vectors really should be 2D column vectors! `@` and solve handle 1D vectors pretty well so we are safe, but be careful.

Computing the inverse is slow - there are faster algorithms when you just want to solve one case, available as `solve` and internally using the LAPACK matrix library. We can even tell solve if we know something special about our matrix, like if we have a diagonal matrix, if we use `scipy.linalg.solve` instead!

In [ ]:
x = np.linalg.solve(A, b)

In [ ]:
A @ x - b

In [ ]:
x

In [ ]:
A = np.array([[4, -2, 1], [3, 6, -4], [2, 1, 8]])

In [ ]:
np.linalg.inv(A) - 1 / 263 * np.array([[52, 17, 2], [-32, 30, 19], [-9, -8, +30]])

In [ ]:
x1 = np.array([12, -25, 32])
x2 = np.array([4, -10, 22])
x3 = np.array([20, -30, 40])

In [ ]:
print(np.linalg.inv(A) @ x1)
print(np.linalg.inv(A) @ x2)
print(np.linalg.inv(A) @ x3)

In [ ]:
print(np.linalg.solve(A, x1))
print(np.linalg.solve(A, x2))
print(np.linalg.solve(A, x3))

For such a tiny problem, `inv` beats `solve` by a hair. But if you invert only once, you can solve many problems with the same solution!

In [ ]:
%%timeit
np.linalg.solve(A, x1)
np.linalg.solve(A, x2)
np.linalg.solve(A, x3)

In [ ]:
%%timeit
np.linalg.inv(A) @ x1
np.linalg.inv(A) @ x2
np.linalg.inv(A) @ x3

In [ ]:
%%timeit
Ai = np.linalg.inv(A)
Ai @ x1
Ai @ x2
Ai @ x3

### Problem: Hilbert matrix

Now let's look at problem 5 in 8.4.3 in our book. For now, let's do it on an 8x8 matrix; on your own try 100x100! We need the Hilbert matrix, which we can find in SciPy:

In [ ]:
a = scipy.linalg.hilbert(8)

Or, given the formula in the book we could have produced it ourselves:

In [ ]:
i, j = np.ogrid[1 : len(a) + 1, 1 : len(a) + 1]
a_ours = 1 / (i + j - 1)

In [ ]:
np.all(a == a_ours)

We need `b`, which is just the first row of the matrix:

In [ ]:
b = a[0]

Let's try solve. When you try 100x100, does this still work?

In [ ]:
np.linalg.solve(a, b)

We can use the invhilbert function to make an inverse hilbert matrix. You can pass `exact=True` to return integers instead of double floats. Note that this matrix will overflow 64 bit integers at 14x14, and therefore will become an inefficient python integer array.

In [ ]:
scipy.linalg.invhilbert(8) @ b

We can also take the inverse ourselves. What happens when this becomes larger?

In [ ]:
np.linalg.inv(a) @ b

## Other tools are available: Eigen Vectors

In [ ]:
I = 1 / 12 * np.array([[8, -3, -3], [-3, 8, -3], [-3, -3, 8]])

In [ ]:
λs, ωs = np.linalg.eig(I)

In [ ]:
λs

In [ ]:
ωs

In [ ]:
ω = ωs[:, 0]
print("ω:", ω)
print("Iω", I @ ω)
print("λω", λs[0] * ω)

In [ ]:
print(I @ ωs - λs * ωs)

Mathematics tends to think of stacked vectors as column vectors - which is the opposite of what's natural to write in a text file, like `[[row1], [row2]]`. This is why matrix oriented languages tend to be column major. You can stack in rows, use `.T` when needed for matrix manipulations. You might want to start in  "F" order if performance is important.